In [10]:
from math import sqrt
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

# Load the dataset
file_path = 'C:\\Users\\loydt\\Downloads\\Projects\\Superstore Sales Dataset.xlsx'
data = pd.read_excel(file_path)

# Convert the 'Order Date' column to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d/%m/%Y', errors='coerce')

# Set 'Order Date' as the index
data.set_index('Order Date', inplace=True)

# List of unique sub-categories
sub_categories = ['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage', 'Furnishings', 
                  'Art', 'Phones', 'Binders', 'Appliances', 'Paper', 'Accessories', 
                  'Envelopes', 'Fasteners', 'Supplies', 'Machines', 'Copiers']

# Initialize an empty DataFrame to store monthly results
monthly_results = pd.DataFrame()

# Loop through each sub-category to calculate seasonal decomposition
for sub_category in sub_categories:
    # Filter data for the current sub-category
    sub_category_data = data[data['Sub-Category'] == sub_category]
    
    # Resample the data to get monthly sales
    monthly_sales = sub_category_data['Sales'].resample('ME').sum()
    
    # Calculate the monthly average sales based on occurrences
    monthly_counts = sub_category_data['Sales'].resample('ME').count()  # Count data points for each month
    monthly_average_sales = monthly_sales / monthly_counts  # Average sales for each month

    # Perform seasonal decomposition
    decomposition = seasonal_decompose(monthly_sales, model='additive', period=12)  # Assuming monthly data
    
    # Extract trend, seasonal, and residual components
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    # Calculate Expected Sales and Performance Deviation
    expected_sales = (trend - np.abs(seasonal)) # Corrected calculation for Expected Sales

    # To ensure expected_sales results are all positive
    #positive_results = np.abs(expected_sales)

    performance_deviation = (residual / expected_sales) * 100  # Corrected term

    # Create a DataFrame for the monthly results
    sub_category_monthly_results = pd.DataFrame({
        'Order Date': monthly_sales.index,  # Include Order Date
        'Total Sales': monthly_sales,
        'Average Monthly Sales': monthly_average_sales,  # Updated to use calculated average
        'Trend': trend,
        'Seasonal': seasonal,
        'Residual': residual,
        'Expected Sales': expected_sales,
        'Performance Deviation (%)': performance_deviation
    })

    # Add the sub-category as a new column for identification
    sub_category_monthly_results['Sub-Category'] = sub_category
    
    # Filter to include only rows with non-null residual values
    valid_results = sub_category_monthly_results[sub_category_monthly_results['Residual'].notna()]
    
    # Append the valid results to the main results DataFrame
    monthly_results = pd.concat([monthly_results, valid_results], ignore_index=True)

# Reset index for better readability
monthly_results.reset_index(drop=True, inplace=True)

# Display the final monthly results with valid values
print(monthly_results)


    Order Date  Total Sales  Average Monthly Sales        Trend     Seasonal  \
0   2015-07-31    1487.6730             743.836500  1702.839492  -117.200645   
1   2015-08-31     794.2760             397.138000  1747.785850  -698.696595   
2   2015-09-30    2394.4698             478.893960  1781.928783  4140.995774   
3   2015-10-31     616.9980             616.998000  1827.875508 -1075.097784   
4   2015-11-30    7263.7137             807.079300  1904.683738  4388.031677   
..         ...          ...                    ...          ...          ...   
600 2018-02-28       0.0000                    NaN  5077.442083 -3174.058897   
601 2018-03-31   21319.8220            3553.303667  5031.611167  5384.325020   
602 2018-04-30       0.0000                    NaN  4449.119167 -1959.512175   
603 2018-05-31    3359.9520            3359.952000  4426.622000  1581.033492   
604 2018-06-30       0.0000                    NaN  5066.618000 -3492.539008   

         Residual  Expected Sales  Perf